# 💀 Bone Fracture Multi-Region X-ray

이 데이터셋은 하지, 상지, 요추, 엉덩이, 무릎 등 모든 해부학적 신체 부위를 포함하는 골절 및 비골절 X-레이 이미지를 포함하고 있습니다. 

데이터 출처 : https://www.kaggle.com/datasets/bmadushanirodrigo/fracture-multi-region-x-ray-data/data


In [76]:
# import
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import torch
from torch import nn #모델 정의
import torch.nn.functional as F #모델 정의
from torch.utils.data import TensorDataset, DataLoader #데이터로드
from sklearn.preprocessing import LabelEncoder
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim # 옵티마이저
from sklearn.metrics import precision_score, recall_score, f1_score

## 데이터 사전 정리

In [81]:
# 파일명 정리
def chage_name(folder_path, name_code, start_numder = 1):
    name_list = [file for file in os.listdir(folder_path) if file.endswith((".png",".jpg",".jpeg"))]
    digit = len(str(len(name_list))) + 1
    for idex, file in enumerate(name_list):
        numder = digit - len(str(idex))
        position = file.rfind(".")
        following_text = file[position:]

        old_file_path = os.path.join(folder_path, file)
        new_file_name = name_code + "0" * numder + str(idex + start_numder) + following_text
        new_file_path = os.path.join(folder_path, new_file_name)
        os.rename(old_file_path, new_file_path)

    updated_files = os.listdir(folder_path)
    print(updated_files)

In [86]:
# 파일 확정자 변경
def change_file_determiner(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".png") or filename.endswith(".bmp"):
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path)
            # 팔레트 모드(P)에서 RGB로 변환
            if img.mode == 'P':
                img = img.convert('RGB')
            # RGBA를 RGB로 변환, 필요한 경우
            elif img.mode == 'RGBA':
                img = img.convert('RGB')
            new_filename = filename.split('.')[0] + '.jpeg'
            new_img_path = os.path.join(directory, new_filename)
            img.save(new_img_path, 'JPEG')
            os.remove(img_path)  # 원본 파일 삭제

In [87]:
# F_name = "BXF_TR_"
# image_path_fractured = "/Users/bumsoojoe/Desktop/Project/Bone Fracture DL/Data/Bone_Fracture_Binary_Classification/train/fractured"

# N_name = "BXN_TR_"
# image_path_No = "/Users/bumsoojoe/Desktop/Project/Bone Fracture DL/Data/Bone_Fracture_Binary_Classification/train/not fractured"

# change_file_determiner(image_path_fractured)
# chage_name(image_path_fractured,F_name,0)
# change_file_determiner(image_path_No)
# chage_name(image_path_No,N_name,0)


# F_name = "BXF_VA_"
# image_path_fractured = "/Users/bumsoojoe/Desktop/Project/Bone Fracture DL/Data/Bone_Fracture_Binary_Classification/val/fractured"

# N_name = "BXN_VA_"
# image_path_No = "/Users/bumsoojoe/Desktop/Project/Bone Fracture DL/Data/Bone_Fracture_Binary_Classification/val/not fractured"

# change_file_determiner(image_path_fractured)
# chage_name(image_path_fractured,F_name,0)
# change_file_determiner(image_path_No)
# chage_name(image_path_No,N_name,0)

# F_name = "BXF_TE_"
# image_path_fractured = "/Users/bumsoojoe/Desktop/Project/Bone Fracture DL/Data/Bone_Fracture_Binary_Classification/test/fractured"

# N_name = "BXN_TE_"
# image_path_No = "/Users/bumsoojoe/Desktop/Project/Bone Fracture DL/Data/Bone_Fracture_Binary_Classification/test/not fractured"

# change_file_determiner(image_path_fractured)
# chage_name(image_path_fractured,F_name,0)
# change_file_determiner(image_path_No)
# chage_name(image_path_No,N_name,0)

['BXF_TR_02935.jpg', 'BXF_TR_01582.jpg', 'BXF_TR_00844.jpg', 'BXF_TR_03395.jpg', 'BXF_TR_03381.jpg', 'BXF_TR_00688.jpg', 'BXF_TR_01596.jpg', 'BXF_TR_00850.jpg', 'BXF_TR_02921.jpg', 'BXF_TR_04388.jpg', 'BXF_TR_02909.jpg', 'BXF_TR_00878.jpg', 'BXF_TR_00139.jpg', 'BXF_TR_01227.jpg', 'BXF_TR_03430.jpg', 'BXF_TR_03356.jpg', 'BXF_TR_04439.jpg', 'BXF_TR_02048.jpg', 'BXF_TR_01541.jpg', 'BXF_TR_00887.jpg', 'BXF_TR_01555.jpg', 'BXF_TR_00893.jpg', 'BXF_TR_03342.jpg', 'BXF_TR_03424.jpg', 'BXF_TR_01233.jpg', 'BXF_TR_02712.jpg', 'BXF_TR_04363.jpg', 'BXF_TR_00105.jpg', 'BXF_TR_00663.jpg', 'BXF_TR_04405.jpg', 'BXF_TR_02074.jpg', 'BXF_TR_04411.jpg', 'BXF_TR_02060.jpg', 'BXF_TR_01569.jpg', 'BXF_TR_00677.jpg', 'BXF_TR_00111.jpg', 'BXF_TR_02706.jpg', 'BXF_TR_04377.jpg', 'BXF_TR_03418.jpg', 'BXF_TR_03197.jpg', 'BXF_TR_02289.jpg', 'BXF_TR_01780.jpg', 'BXF_TR_01958.jpg', 'BXF_TR_03829.jpg', 'BXF_TR_01794.jpg', 'BXF_TR_04335.jpeg', 'BXF_TR_03183.jpg', 'BXF_TR_03815.jpg', 'BXF_TR_03801.jpg', 'BXF_TR_01970.jpg'

/Users/bumsoojoe/anaconda3/envs/myenv/lib/python3.8/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


['BXF_VA_0225.jpg', 'BXF_VA_0231.jpg', 'BXF_VA_0118.jpeg', 'BXF_VA_0027.jpg', 'BXF_VA_0033.jpg', 'BXF_VA_0190.jpg', 'BXF_VA_0184.jpg', 'BXF_VA_0185.jpg', 'BXF_VA_0191.jpg', 'BXF_VA_0146.jpg', 'BXF_VA_0152.jpg', 'BXF_VA_0032.jpg', 'BXF_VA_0071.jpeg', 'BXF_VA_0026.jpg', 'BXF_VA_0230.jpg', 'BXF_VA_0224.jpg', 'BXF_VA_0218.jpg', 'BXF_VA_0226.jpg', 'BXF_VA_0018.jpg', 'BXF_VA_0030.jpg', 'BXF_VA_0024.jpg', 'BXF_VA_0006.jpeg', 'BXF_VA_0178.jpg', 'BXF_VA_0155.jpeg', 'BXF_VA_0144.jpg', 'BXF_VA_0150.jpg', 'BXF_VA_0187.jpg', 'BXF_VA_0179.jpeg', 'BXF_VA_0192.jpg', 'BXF_VA_0186.jpg', 'BXF_VA_0151.jpg', 'BXF_VA_0145.jpg', 'BXF_VA_0025.jpg', 'BXF_VA_0031.jpg', 'BXF_VA_0019.jpg', 'BXF_VA_0180.jpeg', 'BXF_VA_0227.jpg', 'BXF_VA_0233.jpg', 'BXF_VA_0247.jpeg', 'BXF_VA_0223.jpg', 'BXF_VA_0035.jpg', 'BXF_VA_0021.jpg', 'BXF_VA_0009.jpg', 'BXF_VA_0139.jpeg', 'BXF_VA_0206.jpeg', 'BXF_VA_0182.jpg', 'BXF_VA_0196.jpg', 'BXF_VA_0085.jpeg', 'BXF_VA_0103.jpeg', 'BXF_VA_0197.jpg', 'BXF_VA_0168.jpg', 'BXF_VA_0154.jpg', 